<a href="https://colab.research.google.com/github/King-Rian/Project-3/blob/main/project_3_Lou_edits_additions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install emoji library
!pip install emoji
!pip install torch
from google.colab import files
import zipfile
import json
import emoji
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaTokenizer, RobertaModel, AdamW, T5Tokenizer, T5ForConditionalGeneration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 32.9 MB/s eta 0:00:00


In [2]:
# Retrieve the 'emojis.json' zippped file

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Extract the ZIP file
with zipfile.ZipFile('emojis.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Step 3: Load the extracted JSON file
emoji_file = 'emojis.json'

# Using pandas
emoji_df = pd.read_json(emoji_file)
print("Dataset loaded successfully. Sample data:")
print(emoji_df.head())


Saving emojis.json.zip to emojis.json.zip
Dataset loaded successfully. Sample data:
                                            category  \
0  Miscellaneous Symbols And Pictographs -> Emoji...   
1  Miscellaneous Symbols And Pictographs -> Emoji...   
2                                               None   
3  Miscellaneous Symbols And Pictographs -> Emoji...   
4                                               None   

                                            keywords  \
0  [dark skin tone, hand, forbidden, gesture, wom...   
1                     [dark skin tone, woman, guard]   
2                 [racing, running, woman, marathon]   
3  [gymnastics, medium-light skin tone, woman, ca...   
4                                      [woman, golf]   

                                          definition  \
0  The Woman Gesturing Not OK, Type-6 emoji is a ...   
1  The Female Guard, Type-6 emoji is a sequence o...   
2  The female version of the ?? Runner emoji. The...   
3  The Woman Doing

In [3]:
# Check the shape, features, and datatypes of the df
emoji_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    1988 non-null   object
 1   keywords    2389 non-null   object
 2   definition  2389 non-null   object
 3   unicode     2389 non-null   object
 4   name        2389 non-null   object
 5   shortcode   845 non-null    object
 6   senses      2389 non-null   object
dtypes: object(7)
memory usage: 130.8+ KB


In [4]:
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

# Create a new emojis DataFrame with 'name' and 'unicode'
key_value_pairs_df = emoji_df[['name', 'unicode']]

# Process the 'name' column to include : at the beginning and end of the emoji name
key_value_pairs_df['name'] = key_value_pairs_df['name'].apply(
    lambda x: f":{x.split(':')[0].strip()}:"  # Remove everything after the first colon and add colons
)

# Convert the unicode column to emoji symbols
key_value_pairs_df['emoji'] = key_value_pairs_df['name'].apply(emoji.emojize)

# Filter out rows where 'unicode' contains more than one Unicode value
# We check for the number of spaces and ensure it is less than 1
# Eliminating complex emojis and ZWJ sequence values
filtered_df = key_value_pairs_df[key_value_pairs_df['unicode'].str.count(' ') < 1]

# Remove rows where 'emoji' contains text with a beginning and ending ':' - which means that emoji was not found for that :name:
filtered_df = key_value_pairs_df[~key_value_pairs_df['emoji'].str.match(r'^:.*:$')]

# Create a new df and rename features to read for concatination with the next data set
new_filtered_df = filtered_df[['name', 'emoji']]
new_filtered_df = new_filtered_df.rename(columns={'name': 'label'})

# Display the DataFrame with emoji symbols
new_filtered_df.head()

<ipython-input-4-5759284d03f7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_value_pairs_df['name'] = key_value_pairs_df['name'].apply(
<ipython-input-4-5759284d03f7>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_value_pairs_df['emoji'] = key_value_pairs_df['name'].apply(emoji.emojize)


,label,emoji
15,:squid:,🦑
16,:shrimp:,🦐
17,:rhinoceros:,🦏
18,:lizard:,🦎
19,:gorilla:,🦍


In [5]:
# Create an emoji dictionary with labels and their Unicode values from https://carpedm20.github.io/emoji/docs/index.html
emoji_dict = {emoji.demojize(e): e for e in emoji.EMOJI_DATA}

# Show how many key value pairs we retrieved from emoji library
records = len(emoji_dict)
print(f"Number of records: {records}")

# Create new df for use in concatenation the two data frames
emoji_dictionary_df = pd.DataFrame(emoji_dict.items(), columns=['label', 'emoji'])
emoji_dictionary_df.head()
#emoji_dictionary_df.info()

Number of records: 3790


,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [6]:
# Concatentate both dataframes / sources and drop the index
combined_emoji_df = pd.concat([emoji_dictionary_df, new_filtered_df], ignore_index=True)

# Show the key pair data frame with training data
combined_emoji_df.head()

,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [7]:
# Remove : character used for outputting emoji (vs. unicode) and replace underscores with spaces
combined_emoji_df['label'] = combined_emoji_df['label'].str.replace(':', '').str.replace('_', ' ')

# Print output
combined_emoji_df.head()


,label,emoji
0,1st place medal,🥇
1,2nd place medal,🥈
2,3rd place medal,🥉
3,AB button (blood type),🆎
4,ATM sign,🏧


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 127374 (\N{NEGATIVE SQUARED AB}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 127975 (\N{AUTOMATED TELLER MACHINE}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 129351 (\N{FIRST PLACE MEDAL}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 129352 (\N{SECOND PLACE MEDAL}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 129353 (\N{THIRD PLACE MEDAL}) missing from current font.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('emoji').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['emoji'].value_counts()
    for x_label, grp in _df_6.groupby('label')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('label')
_ = plt.ylabel('emoji')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['label'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='index', y='label', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 129351 (\N{FIRST PLACE MEDAL}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 129352 (\N{SECOND PLACE MEDAL}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 129353 (\N{THIRD PLACE MEDAL}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 127374 (\N{NEGATIVE SQUARED AB}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 127975 (\N{AUTOMATED TELLER MACHINE

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['emoji'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='emoji', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [8]:
# Augment label data with synomyms
# !pip install nltk==3.8.1
'''
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

def augment_text(phrase):
  words = phrase.split()
  augmented_words = []
  for word in words:
    synonyms = wordnet.synsets(word)
    if synonyms:
        synonym = synonyms[0].lemmas()[0].name()
        augmented_words.append(synonym)
    else:
         augmented_words.append(word)
    return ' '.join(augmented_words)

combined_emoji_df['label'] = combined_emoji_df['label'].apply(augment_text)

combined_emoji_df.head()
'''

"\nimport nltk\nnltk.download('wordnet')\nfrom nltk.corpus import wordnet\n\ndef augment_text(phrase):\n  words = phrase.split()\n  augmented_words = []\n  for word in words:\n    synonyms = wordnet.synsets(word)\n    if synonyms:\n        synonym = synonyms[0].lemmas()[0].name()\n        augmented_words.append(synonym)\n    else:\n         augmented_words.append(word)\n    return ' '.join(augmented_words)\n\ncombined_emoji_df['label'] = combined_emoji_df['label'].apply(augment_text)\n\ncombined_emoji_df.head()\n"

In [9]:
# Step 1: Prepare the Data
# Create a mapping of emojis to indices
emoji_to_index = {emoji: idx for idx, emoji in enumerate(combined_emoji_df['emoji'].unique())}
index_to_emoji = {idx: emoji for emoji, idx in emoji_to_index.items()}

print("Emoji to Index Mapping:")
print(emoji_to_index)

print("\nIndex to Emoji Mapping:")
print(index_to_emoji)

Emoji to Index Mapping:
{'🥇': 0, '🥈': 1, '🥉': 2, '🆎': 3, '🏧': 4, '🅰': 5, '🇦🇫': 6, '🇦🇱': 7, '🇩🇿': 8, '🇦🇸': 9, '🇦🇩': 10, '🇦🇴': 11, '🇦🇮': 12, '🇦🇶': 13, '🇦🇬': 14, '♒': 15, '🇦🇷': 16, '♈': 17, '🇦🇲': 18, '🇦🇼': 19, '🇦🇨': 20, '🇦🇺': 21, '🇦🇹': 22, '🇦🇿': 23, '🔙': 24, '🅱': 25, '🇧🇸': 26, '🇧🇭': 27, '🇧🇩': 28, '🇧🇧': 29, '🇧🇾': 30, '🇧🇪': 31, '🇧🇿': 32, '🇧🇯': 33, '🇧🇲': 34, '🇧🇹': 35, '🇧🇴': 36, '🇧🇦': 37, '🇧🇼': 38, '🇧🇻': 39, '🇧🇷': 40, '🇮🇴': 41, '🇻🇬': 42, '🇧🇳': 43, '🇧🇬': 44, '🇧🇫': 45, '🇧🇮': 46, '🆑': 47, '🆒': 48, '🇰🇭': 49, '🇨🇲': 50, '🇨🇦': 51, '🇮🇨': 52, '♋': 53, '🇨🇻': 54, '♑': 55, '🇧🇶': 56, '🇰🇾': 57, '🇨🇫': 58, '🇪🇦': 59, '🇹🇩': 60, '🇨🇱': 61, '🇨🇳': 62, '🇨🇽': 63, '🎄': 64, '🇨🇵': 65, '🇨🇨': 66, '🇨🇴': 67, '🇰🇲': 68, '🇨🇬': 69, '🇨🇩': 70, '🇨🇰': 71, '🇨🇷': 72, '🇭🇷': 73, '🇨🇺': 74, '🇨🇼': 75, '🇨🇾': 76, '🇨🇿': 77, '🇨🇮': 78, '🇩🇰': 79, '🇩🇬': 80, '🇩🇯': 81, '🇩🇲': 82, '🇩🇴': 83, '🔚': 84, '🇪🇨': 85, '🇪🇬': 86, '🇸🇻': 87, '🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f': 88, '🇬🇶': 89, '🇪🇷': 90, '🇪🇪': 91, '🇸🇿': 92, '🇪🇹': 93, '🇪

In [10]:
# Step 2: Text Encoding
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_data(phrases):
    # T5 expects the input to be formatted as "translate English to emoji: <text>"
    formatted_phrases = ["translate English to emoji: " + phrase for phrase in phrases.tolist()]
    encoded = tokenizer(formatted_phrases, padding=True, truncation=True, return_tensors='pt')
    return encoded['input_ids'], encoded['attention_mask']

input_ids, attention_masks = tokenize_data(combined_emoji_df['label'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
# Step 3: Prepare Emoji Sequences to Indices
combined_emoji_df['emoji_index'] = combined_emoji_df['emoji'].map(emoji_to_index)
target_indices = torch.tensor(combined_emoji_df['emoji_index'].tolist())  # Convert to tensor

In [12]:
# Step 4: Create DataLoader
dataset = TensorDataset(input_ids, attention_masks, target_indices)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

TypeError: summary() missing 1 required positional argument: 'input_size'

In [35]:
# Step 5: Build the Model
class T5EmojiClassifier(nn.Module):
    def __init__(self, num_classes):
        super(T5EmojiClassifier, self).__init__()
        self.t5 = T5ForConditionalGeneration.from_pretrained('t5-small')
        self.num_classes = num_classes

    def forward(self, input_ids, attention_mask):
        outputs = self.t5(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        return outputs # Return the loss for training

# Initialize the model
num_classes = len(emoji_to_index)
model = T5EmojiClassifier(num_classes)


In [36]:
# Step 6: Training the Model
criterion = nn.CrossEntropyLoss()  # Note: This may not be used directly with T5
optimizer = AdamW(model.parameters(), lr=1e-5)  # Use AdamW instead of Adam

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_masks, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_masks)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Loss: 0.6963334395836606
Epoch 2/3, Loss: 0.022476626367317076
Epoch 3/3, Loss: 0.0067402201878894365


In [37]:
next_sequence=model.generate([word_1, word_2, word_3])
output_id=next_sequence
output+=next_sequence.decode()


AttributeError: 'T5EmojiClassifier' object has no attribute 'generate'

Rian Altering Code

In [28]:
outputs.logits.shape

torch.Size([2, 25, 32128])

In [25]:
# Step 7: Evaluate the Model
model.eval()
val_accuracy = 0
total = 0

with torch.no_grad():
    for batch in train_loader:
        input_ids, attention_masks, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_masks)
        print(outputs)

        # compare outputs vs. labels
        _, predicted = torch.max(outputs, 1)
        val_accuracy += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {val_accuracy / total:.4f}")

Seq2SeqLMOutput(loss=tensor(4.0390e-05, device='cuda:0'), logits=tensor([[[ -0.9473,  -3.2559,  -4.1168,  ..., -32.5583, -32.4064, -32.4220],
         [-18.0573, -13.0711,  -9.5703,  ..., -48.9331, -48.9719, -48.8754],
         [-40.5504, -22.8825, -23.6883,  ..., -72.9204, -73.3053, -73.2102],
         ...,
         [ 45.4907,   0.9936,   0.9484,  ..., -16.5477, -16.6741, -16.5829],
         [ 45.3689,   1.0887,   0.9967,  ..., -16.4553, -16.5773, -16.4867],
         [ 44.8807,   1.0053,   0.9295,  ..., -16.5793, -16.6995, -16.6080]],

        [[ -0.7541,  -2.8723,  -3.4927,  ..., -31.8726, -31.7075, -31.7513],
         [-20.6168, -13.5755,  -9.9961,  ..., -51.5639, -51.6336, -51.5356],
         [-41.7552, -22.2081, -23.6018,  ..., -73.3711, -73.7712, -73.6895],
         ...,
         [ 44.6968,   0.9852,  -0.2106,  ..., -17.8796, -18.0234, -17.9968],
         [ 44.7208,   1.1048,  -0.1117,  ..., -17.6816, -17.8207, -17.7949],
         [ 44.3132,   1.0139,  -0.1631,  ..., -17.7950, -1

TypeError: max() received an invalid combination of arguments - got (Seq2SeqLMOutput, int), but expected one of:
 * (Tensor input, *, Tensor out = None)
 * (Tensor input, Tensor other, *, Tensor out = None)
 * (Tensor input, int dim, bool keepdim = False, *, tuple of Tensors out = None)
 * (Tensor input, name dim, bool keepdim = False, *, tuple of Tensors out = None)


In [ ]:
# Step 8: Make Predictions
def predict(model, tokenizer, text, emoji_to_index):
    model.eval()
    formatted_text = f"translate English to emoji: {text}"
    inputs = tokenizer(formatted_text, return_tensors="pt", truncation=True, padding=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        predicted_index = logits.argmax(dim=1).item()

    return index_to_emoji[predicted_index]

# Example prediction
example_text = "happy face"
predicted_emoji = predict(model, tokenizer, example_text, emoji_to_index)
print(f"Input Text: {example_text}, Predicted Emoji: {predicted_emoji}")


In [ ]:
# Step 9: Test Predictions
test_phrases = ["sad", "laughing", "angry", "love", "dancing"]
for phrase in test_phrases:
    predicted_emoji = predict(model, tokenizer, phrase, emoji_to_index)
    print(f"Input: {phrase}, Predicted Emoji: {predicted_emoji}")


In [ ]:
# Step 10: Build out a user interface
!pip install gradio  # Install Gradio for the interface

import gradio as gr

# Define a function for the interface
def emoji_predictor(text):
    try:
        predicted_emoji = predict(model, tokenizer, text, emoji_to_index)
        return f"Input Text: {text}\nPredicted Emoji: {predicted_emoji}"
    except Exception as e:
        return f"Error: {str(e)}"

# Build the Gradio interface
interface = gr.Interface(
    fn=emoji_predictor,
    inputs=gr.Textbox(label="Enter Text"),
    outputs=gr.Textbox(label="Output"),
    title="Emoji Predictor",
    description="Enter a phrase and get the corresponding emoji prediction based on your trained model."
)

# Launch the interface
interface.launch()
